In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모델에 사용할 데이터
- JSON -> Pandas 변환

## products.json

- product_id
- product_name - str
- product_price
- product_inventory - 재고
- category - 0 ~ 7
- discount
- review_star
- product_quanity - 구매수량

In [1]:
# json data load (products.json)
import json

with open('/Users/daehyunkim_kakao/Desktop/Kakao Business (Project)/Kakaotech-18-AI/dummy_data/products.json', 'r') as file:
  products = json.load(file)

print(products)

[{'product_id': 1, 'product_name': '간식_과자_642', 'product_explanation': 'This is a 과자 product in the 간식 category', 'product_image': 'data_image/image_1.JPG', 'product_price': 72135, 'product_inventory': 56, 'category': '간식/과자', 'discount': 22, 'create_time': '2022-11-04 16:14:17', 'update_time': '2024-08-08 20:34:58.984801', 'cart_id': None, 'member_id': 5, 'review_star': 4.34, 'product_quanity': 202}, {'product_id': 2, 'product_name': '간식_과자_780', 'product_explanation': 'This is a 과자 product in the 간식 category', 'product_image': 'data_image/image_2.JPG', 'product_price': 46861, 'product_inventory': 1, 'category': '간식/과자', 'discount': 44, 'create_time': '2023-06-01 13:59:23', 'update_time': '2024-08-08 20:34:58.984823', 'cart_id': None, 'member_id': 9, 'review_star': 2.94, 'product_quanity': 146}, {'product_id': 3, 'product_name': '간식_과자_663', 'product_explanation': 'This is a 과자 product in the 간식 category', 'product_image': 'data_image/image_3.JPG', 'product_price': 11210, 'product_inv

In [2]:
import pandas as pd

products_data = products
products_df = pd.DataFrame(products_data)
products_df

ModuleNotFoundError: No module named 'pandas'

#### Data Type 확인

In [4]:
# 데이터 타입 확인
print("#"*10)
print("데이터 타입 확인")
print("Data Types:\n", products_df.dtypes)

# 범주형 및 수치형 데이터 분리하여 분석
product_categorical_cols = products_df.select_dtypes(include=['object', 'category']).columns
product_numerical_cols = products_df.select_dtypes(include=['int64', 'float64']).columns

print("#"*10)
print("범주형 데이터 분리하여 분석")
print("\nCategorical Columns:\n", product_categorical_cols)

print("#"*10)
print("수치형 데이터 분리하여 분석")
print("\nNumerical Columns:\n", product_numerical_cols)

##########
데이터 타입 확인
Data Types:
 product_id               int64
product_name            object
product_explanation     object
product_image           object
product_price            int64
product_inventory        int64
category                object
discount                 int64
create_time             object
update_time             object
cart_id                 object
member_id                int64
review_star            float64
product_quanity          int64
dtype: object
##########
범주형 데이터 분리하여 분석

Categorical Columns:
 Index(['product_name', 'product_explanation', 'product_image', 'category',
       'create_time', 'update_time', 'cart_id'],
      dtype='object')
##########
수치형 데이터 분리하여 분석

Numerical Columns:
 Index(['product_id', 'product_price', 'product_inventory', 'discount',
       'member_id', 'review_star', 'product_quanity'],
      dtype='object')


## 콘텐츠 기반 필터링(Content-Based Filtering)을 사용한 이유
- 콘텐츠 기반 필터링은 사용자와 항목의 특징(특성)을 분석하여 추천하는 방법입니다. 사용자가 과거에 선호했던 항목들의 특성을 바탕으로 유사한 특성을 가진 항목을 추천합니다.

- 개인화된 추천: 사용자의 과거 행동과 선호도를 반영하여 개인화된 추천을 제공할 수 있습니다.
- 새로운 항목 추천: 협업 필터링의 콜드 스타트 문제를 피할 수 있으며, 사용자가 새로 추가된 항목도 추천받을 수 있습니다.
- 직관적 이해: 사용자가 왜 특정 항목을 추천받았는지 쉽게 이해할 수 있습니다. 특정 특성(예: 가격, 카테고리 등)이 추천 이유가 되기 때문입니다.
콘텐츠 기반 필터링을 사용한 이유
- 데이터 가용성: 제공된 데이터셋에는 제품, 구매자, 주문, 리뷰 등의 상세한 특성이 포함되어 있어, 각 항목의 특성을 분석하기에 적합했습니다.
- 특성 기반 분석: 제품의 가격, 재고, 할인율 등과 같은 구체적인 특성을 기반으로 리뷰 평점과의 관계를 분석하여 추천 모델을 만들 수 있습니다.
- 사용자 맞춤 추천: 사용자의 구매 이력과 제품 특성을 기반으로 유사한 제품을 추천할 수 있습니다.

이 추천 시스템은 회귀 모델 기반의 협업 필터링 방식입니다. 다양한 피처를 사용하여 사용자가 항목에 부여할 평점을 예측하고, 이를 기반으로 사용자가 아직 경험하지 않은 항목을 추천합니다. 이 방식은 사용자의 과거 평점 데이터를 활용하여 미래의 평점을 예측하고, 높은 평점을 받을 가능성이 있는 항목을 추천하는 방식입니다.

#### 데이터 분포 확인

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# 필요한 열만 선택
selected_features = ['product_id', 'product_name', 'product_explanation', 'product_image',
                     'product_price', 'product_inventory', 'category', 'discount',
                     'create_time', 'update_time', 'cart_id', 'member_id',
                     'review_star', 'product_quanity']

products_df = products_df[selected_features]

In [6]:
# 범주형 변수 인코딩
label_encoder = LabelEncoder()
products_df['category'] = label_encoder.fit_transform(products_df['category'])

# 결측값 처리 (리뷰 평점이 없는 행 제거)
products_df = products_df.dropna(subset=['review_star'])

# 사용자-아이템 행렬을 생성합니다.
user_item_matrix = products_df.pivot(index='member_id', columns='product_id', values='review_star').fillna(0)

# 아이템 간 유사도를 계산합니다.
item_similarity = cosine_similarity(user_item_matrix.T)

# 유사도 데이터프레임을 생성합니다.
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

def get_similar_items(product_id, num_items=5):
    if product_id in item_similarity_df.index:
        similar_items = item_similarity_df[product_id].sort_values(ascending=False).head(num_items)
        return similar_items
    else:
        return f"Product ID {product_id} not found in the dataset"

In [7]:
# 데이터셋에 있는 product_id들을 출력합니다.
available_product_ids = item_similarity_df.index.tolist()
print("Available Product IDs:", available_product_ids)

# 특정 제품과 유사한 제품 추천
product_id_to_check = available_product_ids[0]  # 유효한 제품 ID 중 하나 선택
print(f"Similar items to product ID {product_id_to_check}:")
print(get_similar_items(product_id_to_check, 5))

Available Product IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
Similar items to product ID 1:
product_id
1     1.0
75    1.0
65    1.0
66    1.0
52    1.0
Name: 1, dtype: float64


#### Product Price (상품 가격):
- 가장 높은 중요도를 보입니다. 이는 product_price가 review_star 예측에 가장 큰 영향을 미친다는 것을 의미합니다. 즉, 상품 가격이 리뷰 평점에 매우 중요한 역할을 한다고 볼 수 있습니다.

#### Order Price (주문 가격):
- 두 번째로 높은 중요도를 보입니다. 이는 order_price가 리뷰 평점에 상당히 중요한 영향을 미친다는 것을 의미합니다. 주문 시 총 가격이 리뷰 평점에 영향을 줄 수 있습니다.

#### Product Inventory (상품 재고):
- 중간 정도의 중요도를 보입니다. 이는 product_inventory가 리뷰 평점에 어느 정도 영향을 미친다는 것을 의미합니다. 재고 수량이 많거나 적은 것이 사용자의 리뷰 평점에 영향을 줄 수 있습니다.

#### Discount (할인율):
- 중간 정도의 중요도를 보입니다. 이는 discount가 리뷰 평점에 영향을 미친다는 것을 의미합니다. 할인이 있는 경우 리뷰 평점에 긍정적이거나 부정적인 영향을 줄 수 있습니다.

#### Quantity (구매 수량):
- 가장 낮은 중요도를 보입니다. 이는 quantity가 리뷰 평점에 상대적으로 적은 영향을 미친다는 것을 의미합니다. 구매 수량은 리뷰 평점에 덜 중요한 요소일 수 있습니다.

#### 데이터 전처리및 병합 목적
- selected_features 리스트는 데이터셋에서 선택해야 하는 모든 열을 포함하여, 데이터 전처리 및 병합 작업을 위해 사용됩니다.

In [8]:
from sklearn.preprocessing import LabelEncoder

# products_df 데이터프레임이 이미 로드된 상태로 가정합니다

# 필요한 열 선택
selected_features = ['product_id', 'review_star', 'product_price', 'product_inventory', 'category', 'discount', 'product_quanity']

# 범주형 변수 인코딩
label_encoder = LabelEncoder()
products_df['category'] = label_encoder.fit_transform(products_df['category'])

# 필요한 열 선택
filtered_data = products_df[selected_features]

# 결측값 확인
print("결측값 개수:\n", filtered_data.isnull().sum())
filtered_data = filtered_data.dropna()

# 데이터 확인
print("Filtered Data:\n", filtered_data.head())
print("Filtered Data Shape:", filtered_data.shape)


결측값 개수:
 product_id           0
review_star          0
product_price        0
product_inventory    0
category             0
discount             0
product_quanity      0
dtype: int64
Filtered Data:
    product_id  review_star  product_price  product_inventory  category  \
0           1         4.34          72135                 56         0   
1           2         2.94          46861                  1         0   
2           3         3.12          11210                  0         0   
3           4         3.94          69118                 13         0   
4           5         2.05          61117                 96         0   

   discount  product_quanity  
0        22              202  
1        44              146  
2         2               29  
3        31              115  
4        13              374  
Filtered Data Shape: (80, 7)


#### 사용자 기반 추천 시스템 모델

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

features = ['product_price', 'product_inventory', 'discount', 'product_quanity']
target = 'review_star'

# 훈련 데이터와 테스트 데이터 분할
X = filtered_data[features]
y = filtered_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 회귀 모델 훈련
model = LinearRegression()
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 추천 점수 계산
X_all = filtered_data[features]
filtered_data['predicted_review_star'] = model.predict(X_all)

# 결과 확인
print(filtered_data[['product_id', 'predicted_review_star']].head())

Mean Squared Error: 1.5985774838238949
   product_id  predicted_review_star
0           1               2.991878
1           2               2.679022
2           3               3.379395
3           4               2.723153
4           5               3.447498


이 추천 시스템은 회귀 모델 기반의 협업 필터링 방식입니다. 다양한 피처를 사용하여 사용자가 항목에 부여할 평점을 예측하고, 이를 기반으로 사용자가 아직 경험하지 않은 항목을 추천합니다. 이 방식은 사용자의 과거 평점 데이터를 활용하여 미래의 평점을 예측하고, 높은 평점을 받을 가능성이 있는 항목을 추천하는 방식입니다.

- 일단 해야하는건, 추천시스템은 완성. 다만 카테고리별로 나타내줘야함.

#### 카테고리별 평점 기반 추천시스템 - 아니 불안불안 해서 이건 확인해보려고 만든겁니다
- 간식/과자: 0
- 간식/초콜릿-젤리: 1
- 와인-위스키/레드와인: 2
- 와인-위스키/화이트-로제와인: 3
- 유제품/우유_두유: 4
- 유제품/아이스크림: 5
- 정육-가공육/소고기: 6
- 정육-가공육/돼지고기: 7

In [42]:
print(products_df)

0              간식/과자
1          간식/초콜릿-젤리
2        와인-위스키/레드와인
3    와인-위스키/화이트-로제와인
4          유제품/우유_두유
5          유제품/아이스크림
6         정육-가공육/소고기
7        정육-가공육/돼지고기
Name: category, dtype: object


In [45]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 카테고리 고유 값 확인
unique_categories = products_df.unique()
print("Unique Categories:\n", unique_categories)

# 범주형 변수 인코딩
label_encoder = LabelEncoder()
category_encoded = label_encoder.fit_transform(products_df)

# 카테고리 매핑 정보 생성
category_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

# 카테고리 매핑 정보 출력
print("Category Mapping:")
for category, code in category_mapping.items():
    print(f"{category}: {code}")

# 매핑 확인을 위해 DataFrame 생성
mapping_df = pd.DataFrame({
    'Category': label_encoder.classes_,
    'Encoded Value': range(len(label_encoder.classes_))
})

print("\nCategory to Encoded Value Mapping:")
print(mapping_df)

# 인코딩된 카테고리를 포함한 DataFrame 생성
products_df_encoded = pd.DataFrame({
    'category': products_df,
    'category_encoded': category_encoded
})

print("\nProducts DataFrame with Encoded Categories:")
print(products_df_encoded)

# 원하는 형식으로 출력
print("\nFormatted Output:")
for index, row in products_df_encoded.iterrows():
    print(f"{row['category']}: {row['category_encoded']}")


Unique Categories:
 ['간식/과자' '간식/초콜릿-젤리' '와인-위스키/레드와인' '와인-위스키/화이트-로제와인' '유제품/우유_두유'
 '유제품/아이스크림' '정육-가공육/소고기' '정육-가공육/돼지고기']
Category Mapping:
간식/과자: 0
간식/초콜릿-젤리: 1
와인-위스키/레드와인: 2
와인-위스키/화이트-로제와인: 3
유제품/아이스크림: 4
유제품/우유_두유: 5
정육-가공육/돼지고기: 6
정육-가공육/소고기: 7

Category to Encoded Value Mapping:
          Category  Encoded Value
0            간식/과자              0
1        간식/초콜릿-젤리              1
2      와인-위스키/레드와인              2
3  와인-위스키/화이트-로제와인              3
4        유제품/아이스크림              4
5        유제품/우유_두유              5
6      정육-가공육/돼지고기              6
7       정육-가공육/소고기              7

Products DataFrame with Encoded Categories:
          category  category_encoded
0            간식/과자                 0
1        간식/초콜릿-젤리                 1
2      와인-위스키/레드와인                 2
3  와인-위스키/화이트-로제와인                 3
4        유제품/우유_두유                 5
5        유제품/아이스크림                 4
6       정육-가공육/소고기                 7
7      정육-가공육/돼지고기                 6

Formatted Output:
간식/과자:

In [57]:
# 카테고리 인코딩
label_encoder = LabelEncoder()
filtered_data['category_encoded'] = label_encoder.fit_transform(filtered_data['category'])

# 카테고리 매핑 정보 생성
category_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

# 카테고리별로 제품과 예측 별점 출력
def print_category_recommendations():
    for category, code in category_mapping.items():
        category_data = filtered_data[filtered_data['category_encoded'] == code].copy()

        if not category_data.empty:
            # 예측 별점을 소수점 둘째 자리까지 반올림
            category_data.loc[:, 'predicted_review_star'] = category_data['predicted_review_star'].round(2)
            print(f"Category: {category}")
            print(category_data[['product_id', 'predicted_review_star']].sort_values(by='predicted_review_star', ascending=False))
            print("\n")
        else:
            print(f"No data found for category: {category}")

# 카테고리별 추천 제품 출력
print_category_recommendations()

Category: 0
   product_id  predicted_review_star
8           9                   3.78
4           5                   3.45
6           7                   3.40
2           3                   3.38
9          10                   3.17
0           1                   2.99
7           8                   2.85
3           4                   2.72
1           2                   2.68
5           6                   2.66


Category: 1
    product_id  predicted_review_star
16          17                   3.47
19          20                   3.36
15          16                   3.27
14          15                   3.19
12          13                   3.15
11          12                   3.11
17          18                   3.06
13          14                   2.72
18          19                   2.71
10          11                   2.65


Category: 2
    product_id  predicted_review_star
23          24                   3.36
28          29                   3.36
24          25       

In [60]:
# 전체 데이터프레임에서 predicted_review_star를 소수점 둘째 자리까지 반올림
filtered_data['predicted_review_star'] = filtered_data['predicted_review_star'].round(2)

# 병합된 데이터 확인 (반올림 적용 후)
print("Merged Data Preview:")
print(filtered_data.head())

# 병합된 데이터를 CSV 파일로 저장
output_filename = 'Kakao_Recsys_Output.csv'
filtered_data.to_csv(output_filename, index=False)

print(f"Data has been saved to {output_filename}")


Merged Data Preview:
   product_id  review_star  product_price  product_inventory  category  \
0           1         4.34          72135                 56         0   
1           2         2.94          46861                  1         0   
2           3         3.12          11210                  0         0   
3           4         3.94          69118                 13         0   
4           5         2.05          61117                 96         0   

   discount  product_quanity  predicted_review_star  category_encoded  
0        22              202                   2.99                 0  
1        44              146                   2.68                 0  
2         2               29                   3.38                 0  
3        31              115                   2.72                 0  
4        13              374                   3.45                 0  
Data has been saved to Kakao_Recsys_Output.csv


In [ ]:
for product in products:
    product_id = product['product_id']

    # filtered_data에서 해당 product_id의 predicted_review_star 값을 가져오기
    predicted_star = filtered_data.loc[filtered_data['product_id'] == product_id, 'predicted_review_star'].values
    
    if len(predicted_star) > 0:
        product['predicted_review_star'] = predicted_star[0]  # 값을 추가

# 결과를 새로운 JSON 파일로 저장
output_json_filename = 'product_predictions.json'
with open(output_json_filename, 'w', encoding='utf-8') as f:
    json.dump(products_data, f, ensure_ascii=False, indent=4)

print(f"Updated JSON file has been saved to {output_json_filename}")